# Simulator

In [1]:
%%writefile hpdwsimulator.py

from itertools import product
import numpy as np
import copy as copy

def euclidean(source, dest):
    return (source[0]-dest[0])**2 + (source[1]-dest[1])**2 + (source[2]-dest[2])**2

def blksEqual(blk1, blk2):
    if (blk1 == '*' and blk2 != '') or (blk2 == '*' and blk1 != ''):       
        return True   
    else:
        return blk1 == blk2
    
def readFile(Xrange, Zrange, Yrange, filename):
    stateMap = {}
    with open(filename, 'r') as file_in:
            for line in file_in.readlines():
                data = line.split()
                #x, z, y, item = (int(data[0]), int(data[1]), int(data[2]), data[3])   
                x, z, y, item = (data[0], data[1], data[2], data[3])  
                if x != '?':
                    x=int(x)
                if z != '?':
                    z=int(z)
                if y != '?':
                    y=int(y)
                if (x, z) not in stateMap:
                    stateMap[(x, z)] = ['']*(Yrange[1]+1)                    
                stateMap[(x, z)][y] = item
    return stateMap

class HPDWSimulator: 
    # loglevel can be 'verbose' or 'silent'
    def __init__(self, Xrange, Zrange, Yrange, filename, loglevel='silent'):
        self.Xrange = Xrange
        self.Zrange = Zrange
        self.Yrange = Yrange
        self.loglevel = loglevel
        self.attached = False        
        self.dronePos = None
        self.stateMap = readFile(Xrange, Zrange, Yrange, filename)
        for xz, xzItems in self.stateMap.items():
            for y, item in enumerate(xzItems):
                if item == 'd':
                    self.dronePos = (xz[0], xz[1], y)
        
    def __repr__(self):
        representation = "loglevel=" + repr(self.loglevel) + " Xrange=" + repr(self.Xrange) + " Zrange=" + repr(self.Zrange) + \
                         " Yrange=" + repr(self.Yrange) + " attached=" + repr(self.attached) + \
                         " dronePos=" + repr(self.dronePos) + "\nstateMap:\n"
        for columnXZ, columnItems in self.stateMap.items(): 
            countNonEmptyItems = sum(1 for i in columnItems if i != '')
            if  countNonEmptyItems > 0:
                representation = representation + repr(columnXZ) + ">" + repr(columnItems) + "\n"
            
        return representation
    
    def isBlkAt(self, blk, xz, y):
        if xz in self.stateMap and self.numBlksAt(xz) > y:            
            if blk == '*':
                return True
            else:
                return blk == self.blkAt(xz, y)
        else:
            return False
        
    def towerAt(self, xz):
        if xz in self.stateMap:
            return self.stateMap[xz]
        else:
            return []
        
    def towersWithBlks(self):        
        return [(xz, tower) for xz, tower in self.stateMap.items() if self.numBlksAt(xz)>0]
        
    def blkAt(self, xz, y):
        if xz in self.stateMap and self.numBlksAt(xz) > y:            
            return self.stateMap[xz][y]
        else:
            return ''
        
    def topBlkAt(self, xz):
        if xz in self.stateMap:            
            return self.stateMap[xz][self.numBlksAt(xz)]
        else:
            return ''
        
    def numBlksAt(self, xz):
        if xz in self.stateMap:
            return sum(1 for item in self.stateMap[xz] if item != '' and item !='d')
        else:
            return 0
    
    def numBlksAtIncludeEmpty(self, xz):
        if xz in self.stateMap:
            return sum(1 for item in self.stateMap[xz] if item !='d')
        else:
            return 0
    
    def getPossibleXZs(self):
        Xs = range(self.Xrange[0], self.Xrange[1]+1)
        Zs = range(self.Zrange[0], self.Zrange[1]+1)
        XZs=[]
        for x in Xs:
            for z in Zs:
                XZs.append((x,z))
        return XZs
    
    def fillWC(self):
        filledStateMap = {}
        for xz, xzTower in self.stateMap.items():
            Xs = range(self.Xrange[0], self.Xrange[1]+1) if xz[0] == '?' else range(xz[0], xz[0]+1)
            Zs = range(self.Zrange[0], self.Zrange[1]+1) if xz[1] == '?' else range(xz[1], xz[1]+1)
            ht = len(xzTower) - next(i for i, item in enumerate(xzTower[::-1]) if item != 'd' and item != '')
            xzTower= ['*' if item == '' or item == '?' else item for item in xzTower[:ht]]

            for x in Xs:
                for z in Zs:
                    filledStateMap[(x,z)]=xzTower
                    
        self.stateMap = filledStateMap
        return 
    
    def fillWCWithPossibleTowers(self, possibleTowers):
        result = copy.deepcopy(self)        
        filledStateMap ={}
        for (xz, xzTower) in possibleTowers:            
            filledStateMap[xz]=xzTower
                    
        result.stateMap = filledStateMap
        return result
    
    def fillWCInY(self):
        filledStateMap = {}
        for xz, xzTower in self.stateMap.items():
            ht = len(xzTower) - next(i for i, item in enumerate(xzTower[::-1]) if item != 'd' and item != '')
            xzTower= ['*' if item == '' or item == '?' else item for item in xzTower[:ht]]

            filledStateMap[xz]=xzTower
                    
        self.stateMap = filledStateMap
        return 
    
    def getPossibleTowers(self):
        possibleTowers = []
        for xz, xzTower in self.stateMap.items():
            Xs = range(self.Xrange[0], self.Xrange[1]+1) if xz[0] == '?' else range(xz[0], xz[0]+1)
            Zs = range(self.Zrange[0], self.Zrange[1]+1) if xz[1] == '?' else range(xz[1], xz[1]+1)
            ht = len(xzTower) - next(i for i, item in enumerate(xzTower[::-1]) if item != 'd' and item != '')
            xzTower= ['*' if item == '' or item == '?' else item for item in xzTower[:ht]]

            for x in Xs:
                for z in Zs:
                    possibleTowers.append(((x,z), xzTower))
        return possibleTowers

    def log(self, message):
        if self.loglevel is 'verbose':
            print(message)
    
    # returns True for success, else False
    def attach(self):
        if self.attached:
            self.log("Already attached...not attaching!")
            return False
        else:
            self.attached = True
            return True
       
    # return True if success, else False
    def release(self):
        if not self.attached:
            self.log("Not attached...can't release!")
            return False
        else:
            # find top of column below drone
            # first the column of items below drone and attached block
            droneColumnXZ = (self.dronePos[0], self.dronePos[1])
            droneY = self.dronePos[2]
            #attached block at droneY-1 
            attachedBlock = self.stateMap[droneColumnXZ][droneY-1]
#             print("During release, attached block is\n", attachedBlock)
            # now the drone column items below the attached block at droneY-1
            droneColumnItems = self.stateMap[droneColumnXZ][:droneY - 1] 
#             print("During release, droneColumnItems\n", droneColumnItems)
            if '' in droneColumnItems:
                # find first '' (empty) item bottoms up on the remaining column to take in the attached block to be released
                destY = droneColumnItems.index('')
#                 print("During release, first empty slot in droneColumnItems\n", destY)
                # now place attached block at droneY-1 at destY
                self.stateMap[droneColumnXZ][destY] = attachedBlock
                # mark previously attached block position at droneY-1 as '' (empty)
                self.stateMap[droneColumnXZ][droneY-1] = ''
                self.attached = False
                return True
            else:
#                 self.log('Column {} does not have free slots to take the release... not releasing!'.format(droneColumnXZ) )
                return False
  
    def move(self, dx, dz, dy):
        # alt drone pos by dx, dz, dy
        origDronePos = self.dronePos
        self.dronePos = (origDronePos[0]+dx, origDronePos[1]+dz, origDronePos[2]+dy)
        if (self.dronePos[0], self.dronePos[1]) not in self.stateMap:
            self.stateMap[(self.dronePos[0], self.dronePos[1])] = ['']*(self.Yrange[1]+1)
        self.stateMap[(self.dronePos[0], self.dronePos[1])][self.dronePos[2]] = 'd'
        self.stateMap[(origDronePos[0], origDronePos[1])][origDronePos[2]] = ''
        
        # alt attached block position if attached
        if self.attached:
            origPos = (origDronePos[0], origDronePos[1], origDronePos[2]-1)
            newPos = (origPos[0]+dx, origPos[1]+dz, origPos[2]+dy)
            attachedBlock = self.stateMap[(origPos[0], origPos[1])][origPos[2]]
            if (newPos[0], newPos[1]) not in self.stateMap:
                self.stateMap[(newPos[0], newPos[1])] = ['']*(self.Yrange[1]+1)
            self.stateMap[(newPos[0], newPos[1])][newPos[2]] = attachedBlock
            self.stateMap[(origPos[0], origPos[1])][origPos[2]] = ''
            
        return True
       
    def possibleActions(self):
        actions = [] # # Returns tuples of (attach), (release), (move, dx, dz, dy)
        droneXZ = (self.dronePos[0], self.dronePos[1])
        droneX = self.dronePos[0]
        droneZ = self.dronePos[1]
        droneY = self.dronePos[2]        
        droneColumnItems = self.stateMap[droneXZ]        
        # First find possible attaches
        if droneY >= 1 and droneColumnItems[droneY - 1] != '' and (not self.attached) :
            actions.append(('attach',))
        # Now find possible releases
        if droneY >= 1 and '' in droneColumnItems[:droneY - 1] and self.attached:
            actions.append(('release',))
        
        # Finally find possible moves (in ideal world there are 27 such moves involving cartesian products of {-1}, {0}, {1})
        moves = set(product(set([-1,0,1]), repeat=3))
        # Now append to actions if we are allowed to make these moves
        for move in moves:
            dx, dz, dy = move
            # first check if drone move is within drone world ranges
            if (droneX + dx >= self.Xrange[0] and droneX + dx <= self.Xrange[1]) and \
                (droneZ + dz >= self.Zrange[0] and droneZ + dz <= self.Zrange[1]) and \
                (droneY + dy >= self.Yrange[0] and droneY + dy <= self.Yrange[1]) and \
                ((droneX + dx, droneZ + dz) not in self.stateMap or self.stateMap[(droneX + dx, droneZ + dz)][droneY + dy] == ''):
                # if attached, check whether attached block (1 below drone) can also move to corresponding new position
                if self.attached:
                    if (droneY + dy - 1 < self.Yrange[0]) or \
                        (droneY + dy - 1 > self.Yrange[1]) or \
                         ( ((droneX + dx, droneZ + dz) in self.stateMap) and \
                            (self.stateMap[(droneX + dx, droneZ + dz)][droneY + dy - 1] is not '') ) :
                        continue                    
                
                actions.append(('move', dx, dz, dy))
                    
        self.log("Possible actions {}".format(actions))            
        return actions
    
    def possibleDroneMoves(self):
        actions = [] # # Returns tuples of (attach), (release), (move, dx, dz, dy)
        droneXZ = (self.dronePos[0], self.dronePos[1])
        droneX = self.dronePos[0]
        droneZ = self.dronePos[1]
        droneY = self.dronePos[2]        
        droneColumnItems = self.stateMap[droneXZ]        
#         # First find possible attaches
#         if droneY >= 1 and droneColumnItems[droneY - 1] != '' and (not self.attached) :
#             actions.append(('attach',))
#         # Now find possible releases
#         if droneY >= 1 and '' in droneColumnItems[:droneY - 1] and self.attached:
#             actions.append(('release',))
        
        # Finally find possible moves (in ideal world there are 27 such moves involving cartesian products of {-1}, {0}, {1})
        moves = set(product(set([-1,0,1]), repeat=3))
        # Now append to actions if we are allowed to make these moves
        for move in moves:
            dx, dz, dy = move
            # first check if drone move is within drone world ranges
            if (droneX + dx >= self.Xrange[0] and droneX + dx <= self.Xrange[1]) and \
                (droneZ + dz >= self.Zrange[0] and droneZ + dz <= self.Zrange[1]) and \
                (droneY + dy >= self.Yrange[0] and droneY + dy <= self.Yrange[1]) and \
                ((droneX + dx, droneZ + dz) not in self.stateMap or self.stateMap[(droneX + dx, droneZ + dz)][droneY + dy] == ''):
                # if attached, check whether attached block (1 below drone) can also move to corresponding new position
                if self.attached:
                    if (droneY + dy - 1 < self.Yrange[0]) or \
                        (droneY + dy - 1 > self.Yrange[1]) or \
                         ( ((droneX + dx, droneZ + dz) in self.stateMap) and \
                            (self.stateMap[(droneX + dx, droneZ + dz)][droneY + dy - 1] is not '') ) :
                        continue                    
                
                actions.append(('move', dx, dz, dy))
                    
        self.log("Possible actions {}".format(actions))            
        return actions
    
    def takeActionImmutable(self, action):
        resultingDWSim = copy.deepcopy(self)
        (actionStatus, stepCost) = resultingDWSim.takeAction(action)
        return (actionStatus, stepCost, resultingDWSim)
        
    # assumes a valid action returned by possibleActions
    def takeAction(self, action):
        # returns a tuple of True/False depending on success/failure status of action and step cost.
        # step cost is euclidean for move, 1 for release/attach        
        status = False
        stepCost = -1.0
        self.log("Taking action {}".format(action))
        if action[0] == 'attach':
            status = self.attach()
            stepCost = 1.0
        elif action[0] == 'release':
            status = self.release()
            stepCost = 1.0
        elif action[0] == 'move':
            actionStr, dx, dz, dy = action
            status = self.move(dx, dz, dy)
            source = self.dronePos
            dest = (self.dronePos[0]+dx, self.dronePos[1]+dz, self.dronePos[2]+dy)
            stepCost = euclidean(source, dest)
        else:
            self.log('Invalid action requested!')            
        return (status, stepCost)
    
    def isGoal(self, goal):
        diffs = 0
        for goalTowerXZ, goalTowerItems in goal.stateMap.items():
            if (self.numBlksAt(goalTowerXZ) == goal.numBlksAt(goalTowerXZ)):
                towerItems = self.towerAt(goalTowerXZ)
                diffs = diffs + sum(1 for i, item in enumerate(towerItems[:self.numBlksAt(goalTowerXZ)]) \
                                    if (not blksEqual(item, goalTowerItems[i])) )
            else:
                return False        
        return diffs==0
   
    def moveDroneImmutable(self, droneDest):
        resultingState = copy.deepcopy(self)
        origDronePos = resultingState.dronePos
        resultingState.dronePos = droneDest
        if (resultingState.dronePos[0], resultingState.dronePos[1]) not in resultingState.stateMap:
            resultingState.stateMap[(resultingState.dronePos[0], resultingState.dronePos[1])] = \
                                                                   ['']*(resultingState.Yrange[1]+1)
            
        resultingState.stateMap[(resultingState.dronePos[0], resultingState.dronePos[1])][resultingState.dronePos[2]] = 'd'
        resultingState.stateMap[(origDronePos[0], origDronePos[1])][origDronePos[2]] = ''
        
        # alt attached block position if attached
        if resultingState.attached:
            origPos = (origDronePos[0], origDronePos[1], origDronePos[2]-1)
            newPos = (droneDest[0], droneDest[1], droneDest[2]-1)
            attachedBlock = resultingState.stateMap[(origPos[0], origPos[1])][origPos[2]]
            if (newPos[0], newPos[1]) not in resultingState.stateMap:
                resultingState.stateMap[(newPos[0], newPos[1])] = ['']*(resultingState.Yrange[1]+1)
            resultingState.stateMap[(newPos[0], newPos[1])][newPos[2]] = attachedBlock
            resultingState.stateMap[(origPos[0], origPos[1])][origPos[2]] = ''
        
        return resultingState
    
def goalTest(dwsim, goal):
    return dwsim.isGoal(goal)

Overwriting hpdwsimulator.py


# HLAs

In [2]:
%%writefile hla.py
from copy import *
from hpdwsimulator import *

class Plan:
    def __init__(self, startState=None, endState=None):
        self.startState = startState
        self.endState = endState        
        self.actions = []
    
    def __repr__(self):
        output =''
        for i, action in enumerate(self.actions):
            if i > 0:
                output += "," + repr(action)
            else:
                output += repr(action)
        
        return output
    
    def addAction(self, action):
        self.actions.append(action)
    
    def getPlanCost(self):
        return sum(action.getBestPlanCost() for action in actions)

class A:
    def __init__(self, name, startState):
        self.name = name
        self.startState = startState
        self.endState = None        
        self.possiblePlans = [] # possible plans sorted by cost
        self.cost = 0 
        
    def __repr__(self, name):
        return "A:" + repr(name)  
    
    def evalPlans(self):
        return
    
    def addPlan(self, possiblePlan):
        self.possiblePlans.append(possiblePlan)
        self.possiblePlans.sort(key = lambda n: n.getPlanCost()) # sort by cost of plan
        
    def getBestPlanCost(self):
        return 0
   
    
class BuildTower(A):
    def __init__(self, startState, xz, tower):
        A.__init__(self, name="BuildTower", startState=startState)
        self.xz = xz
        self.tower = tower # list of blocks - where each block can be either a colore or '*' meaning any color
    
    def __repr__(self):
        return self.name + " " + repr(self.tower) + " at " + repr(self.xz) 
    
    def evalPlans(self):
        # eval plans required to achieve this, cost and endstate, set them on base action. 
        # based on start state find how many blocks to move to dest tower        
        plans=[]
        currentTower = self.startState.towerAt(self.xz)
        currentTowerBlkCnt = self.startState.numBlksAt(self.xz)
        moreBlksRequiredInTower = len(self.tower)        
        
        # chk from bottom the first unequal blk in current tower - let's say it's tower[firstuneuqalidx]
        # from top (in a loop) move out i.e. MoveBlk(startState=startState, srcxz=xz, destxz=('?','?') 
        # out of tower up unitl firstuneuqalidx
        
        firstunequalidx = next( (i for i, currentBlk in enumerate(currentTower[:currentTowerBlkCnt]) \
                                 if not blksEqual(currentBlk, self.tower[i])), None)
        moveoutHlas = []
        if firstunequalidx is not None:
            print(self)
            print("currentTower", currentTower)
            print("first unequal Idx", firstunequalidx)
            [moveoutHlas.append(MoveBlk(startState=self.startState, srcxz=self.xz, destxz=('?','?') )) \
            for i, blk in enumerate(currentTower[:currentTowerBlkCnt][::-1]) if i >= firstunequalidx]
                
        # 2 strategies for plans here on:
        
        # 1: Now go again in a loop from idx=firstuneuqalidx upwards and bring self.tower[idx] 
        #    MoveBlkIn(starstState=startState, toxz=self.towerxz, self.tower[idx]) 
        #    any srcxz which has at least one block that is equal to self.tower[idx]; idx++
        
        # 2: Just get the blocks from any tower with blocks having anyone of the desired blocks in self.tower
        #    if blocks not in order, do recursively buildTower again - hope is moveoouts followed by moveIns successively 
        #    will make it proper at some stage
        
        # for each remaining block, explore all possible srcxz out of a other towers - each possiblity leads to a different plan
        plan = Plan(startState=self.startState)
        for hla in moveoutHlas:
            plan.addAction(hla)
            
        plans.append(plan)
        
        print(plans)
        
        return

class MoveBlkIn:
    def __init__(self, startState, destxz, blk):
        A.__init__(self, name="MoveBlkIn", startState=startState)
        self.destxz = destxz
        self.blkColor = blkColor
        
    def __repr__(self):
        return "MoveBlkIn:" + repr(self.blk) + "->" + repr(self.destxz)
    
    def evalPlans(self):
        # eval plans required to achieve this, cost and endstate, set them on base action.
        # will consist of a bunch of MoveBlk HLAs
        #    identify such srcxz towers with desired block
        #    for each such srcxz:
        #       recurse:
        #           if top block != desired block (self.blk), move out top i.e. MoveBlk(startState, srcxz, destxz=('?','?') )
        #       
        #       move top block to toxz i.e. MoveBlk(startState, srcxz, destxz=toxz) until 
        #    for all such towers
        return
        
class MoveBlk:
    def __init__(self, startState, srcxz, destxz=('?','?')):
        A.__init__(self, name="MoveBlk", startState=startState)
        self.srcxz=srcxz
        self.destxz=destxz   
        
    def __repr__(self):
        return "MoveBlk:" + repr(self.srcxz) + "->" + repr(self.destxz)
    
    def evalPlans(self):
        # eval plans required to achieve this, cost and endstate, set them on base action.
        # drone has to move to top of srcxz, attach, move to top of destxz, release
        plans = []
        plan = Plan(startState=self.startState)
        cost = 0
        # move to top of srcxz
        x = self.srcxz[0]
        z = self.srcxz[1]
        y = self.startState.numBlksAt(self.srcxz)
        moveToSrcTop = MoveDrone(startState=self.startState, droneDest=(x,z,y))
        moveToSrcTop.evalPlans()
        cost = cost + moveToSrcTop.cost
        plan.addAction(moveToSrcTop)
#         print("After {} with cost {} end state is \n {}".format(moveToSrcTop, moveToSrcTop.cost, moveToSrcTop.endState))
        
        attachToTopOfSrc = Attach(startState=moveToSrcTop.endState)
        attachToTopOfSrc.evalPlans()
        cost = cost + attachToTopOfSrc.cost
        plan.addAction(attachToTopOfSrc)
#         print("After {} with cost {} end state is \n {}".format(attachToTopOfSrc, attachToTopOfSrc.cost, attachToTopOfSrc.endState))  
        
        x = self.destxz[0]
        z = self.destxz[1]
        #y = self.startState.numBlksAt(self.destxz) if self.startState.numBlksAt(self.destxz)>0 else self.startState.Yrange[1]
        y = self.startState.Yrange[1]
        moveToDestTop = MoveDrone(startState=attachToTopOfSrc.endState, droneDest=(x,z,y))
        moveToDestTop.evalPlans()
        cost = cost + moveToDestTop.cost
        plan.addAction(moveToDestTop)
#         print("After {} with cost {} end state is \n {}".format(moveToDestTop, moveToDestTop.cost, moveToDestTop.endState))
        
        releaseFromTopOfDest = Release(startState=moveToDestTop.endState)
        releaseFromTopOfDest.evalPlans()
        cost = cost + releaseFromTopOfDest.cost
        plan.addAction(releaseFromTopOfDest)
#         print("After {} with cost {} end state is \n {}".format(releaseFromTopOfDest, releaseFromTopOfDest.cost, releaseFromTopOfDest.endState))
        
        self.endState = releaseFromTopOfDest.endState
        self.cost = cost
#         print("total cost is ", self.cost)
        plans.append(plan)
        
        return

# A* will be required for shorted path for MoveDrone
class MoveDrone:
    def __init__(self, startState, droneDest):
        A.__init__(self, name="MoveDrone", startState=startState)
        
        self.droneDest = droneDest   
        
    def __repr__(self):
        return "MoveDrone" + "->" + repr(self.droneDest)
    
    def evalPlans(self):
        # eval plans required to achieve this, cost and endstate, set them on base action.
        # drone has to move to from current position to top of destxz
        # returns resulting state
        self.endState = self.startState.moveDroneImmutable(self.droneDest)
        
        source = self.startState.dronePos
        dest = self.endState.dronePos        
        self.cost = euclidean(source, dest)
        
        return 

class Attach(A):
    def __init__(self, startState):
        A.__init__(self, name='attach', startState=startState)            
        
    def __repr__(self):
        return self.name
    
    def evalPlans(self):
        (actionStatus, stepCost, resultingState) = self.startState.takeActionImmutable(('attach',))
        self.endState = resultingState
        self.cost = stepCost
        return 

class Release(A):
    def __init__(self, startState):
        A.__init__(self, name='release', startState=startState)            
        
    def __repr__(self):
        return self.name
    
    def evalPlans(self):
        (actionStatus, stepCost, resultingState) = self.startState.takeActionImmutable(('release',))
        self.endState = resultingState
        self.cost = stepCost
        return 
      
    
class PA(A):
    def __init__(self, startState, actionTuple):
        A.__init__(self, name='PA', startState=startState)
        self.actionTuple = actionTuple
        plan = Plan()
        plan.addAction(self)
        self.possiblePlans.append(plan)       
        
    def __repr__(self):
        return repr(self.actionTuple)
    
    def evalPlans(self):
        # eval endState for this action and set it on base high level A.endState
        return
        
    def getBestPlanCost(self):
        #TODO calculate cost of this primitive action and return, overrides high level Action cost
        return 0            
    
def getPossiblePlansFromGoal(startState, goal):
    #TODO: see if need to readjust if more than 1 tower to be built, goal contradiction etc.
    #TODO: At least let's create plans with all permutations of build towers
    plans = []
    for possibletower in goal.getPossibleTowers():
        plan = Plan(startState=startState)
        plan.addAction(BuildTower(startState=startState, xz=possibletower[0], tower=possibletower[1]))
        plans.append(plan)          
    return plans



# print(getPossibleTowers(state3, goal3))
    

Overwriting hla.py


# HP Heuristics

In [3]:
%%writefile hpdwheuristics.py

from hpdwsimulator import *
from itertools import product
import numpy as np

def euclidean(source, dest):
    return (source[0]-dest[0])**2 + (source[1]-dest[1])**2 + (source[2]-dest[2])**2

def manhattan(source, dest):
    return abs(source[0]-dest[0]) + abs(source[1]-dest[1]) + abs(source[2]-dest[2])

def moveCostToDronePos(state, destDronePos):
    source = state.dronePos    
    return euclidean(source, destDronePos)

def moveDroneCost(currentState, goalState):
    source = currentState.dronePos
    dest = goalState.dronePos
    productOfXZDimensions = ( abs(source[0]-dest[0]) ) * ( abs(source[1]-dest[1]) )
    return  (source[0]-dest[0])**2 + (source[1]-dest[1])**2 + (source[2]-dest[2])**2 + \
            (source[0]-dest[0])**2 + (source[1]-dest[1])**2 + \
            (source[0]-dest[0])**2 + (source[2]-dest[2])**2 + \
            (source[1]-dest[1])**2 + (source[2]-dest[2])**2 + \
            euclidean(source, dest)
            
def moveBlocksHLACostDebug(currentState, goalState):
    if currentState.isGoal(goalState):
        return 0
    
    cost = 0
    #Current state has a blank and goal state has a block. 
    #So increment the heuristic value by the number of blocks needed to be moved into a place
    
    for goalxz, goaltower in goalState.stateMap.items():
        if currentState.numBlksAt(goalxz) == 0:
            cost = cost + goalState.numBlksAt(goalxz)
            
    print("After crtieria 1 cost= ", cost)        
    #Current state has a block and goal state has a blank. 
    #So increment the heuristic value by the number of blocks needed to be moved out of a place.

    for currentxz, currenttower in currentState.stateMap.items():
        if currentState.numBlksAt(currentxz) > 0 and goalState.numBlksAt(currentxz) == 0:
            cost = cost + currentState.numBlksAt(currentxz)
            
    print("After crtieria 2 cost= ", cost)
    
    #Both states have a block, but they are not the same. The incorrect block must be moved out 
    #and the correct blocks must be moved in. 
    #Increment the heuristic value by the number of these blocks. 
    
    for goalxz, goaltower in goalState.stateMap.items():
        if currentState.numBlksAtIncludeEmpty(goalxz) > 0:
            currenttower = currentState.towerAt(goalxz)
            currentEmptyCntNeedsFill=0
            currentNonEmptyCntNeedsChange=0
            for goalBlkIdx, goalBlk in enumerate(goaltower):
                if (blksEqual(goalBlk, currenttower[goalBlkIdx]) == False):
                    if currenttower[goalBlkIdx] == '':                        
                        currentEmptyCntNeedsFill = currentEmptyCntNeedsFill + 1
                    else:
                        currentNonEmptyCntNeedsChange = currentNonEmptyCntNeedsChange + 2
            
            cost = cost + currentEmptyCntNeedsFill
            cost = cost + currentNonEmptyCntNeedsChange
            
    print("After crtieria 3 cost= ", cost)   
    
    #Both states have a block, and they are the same. However, the position above it is incorrect. 
    #Increment the heuristic value by the number of incorrect positions above the correct block.
    
#     for goalxz, goaltower in goalState.stateMap.items():
#         currentTowerBlkCnt = currentState.numBlksAtIncludeEmpty(goalxz)
#         goalTowerBlkCnt = goalState.numBlksAt(goalxz)
#         if (currentTowerBlkCnt > 0 and currentTowerBlkCnt >= goalTowerBlkCnt):            
#             currenttower = currentState.towerAt(goalxz)
            
#             firstunequalidx = next( (i for i, currentBlk in enumerate(currenttower[:goalTowerBlkCnt]) \
#                                  if (not blksEqual(currentBlk, goaltower[i])) ), None)
            
#             if firstunequalidx is not None:
#                 cost = cost + (goalTowerBlkCnt - firstunequalidx) 
                
#     print("After crtieria 4 cost= ", cost)
    
    
    # For a given (nonempty) place, find the top-most block. Look for its position in the goal state. 
   
    for currentxz, currenttower in currentState.stateMap.items():
            currentTowerBlkCnt = currentState.numBlksAt(currentxz)
            if currentTowerBlkCnt > 0:
                currentTowerTopBlk = currenttower[currentTowerBlkCnt-1]
                print("currenttower at {} {}".format(currentxz, currenttower))            
                for goalxz, goaltower in goalState.stateMap.items():
                    print("goaltower at {} {}".format(goalxz, goaltower)) 
                    goalTowerBootomBlk = goaltower[0]
                    goalTowerBlkCnt = goalState.numBlksAt(currentxz)
                    if ( goalxz != currentxz ):
                        if blksEqual(goalTowerBootomBlk, currentTowerTopBlk) :
                        #If tower top item goes on the bottom row and the place is currently empty, the block can be moved there, 
                        #so decrement heuristic by one.
                            print("Decrement by 1 as top of currenttower equals bottom of goaltower")
                            cost = cost - 1
                        else: 
                        #else if the top-most block doesn't go on bottom row, but all blocks below it in the goal state are 
                        #currently in their correct place, then the block can be moved there, so decrement heuristic by one.
                            print("currentTowerBlkCnt=", currentTowerBlkCnt)
                            print("goalTowerBlkCnt=", goalTowerBlkCnt)
                            if (currentTowerBlkCnt > 1):
                                s=sum(1 for i, currentTowerBlk in enumerate(currenttower[:currentTowerBlkCnt-1]) \
                                      if (blksEqual(currentTowerBlk, goaltower[i])==False) )
                                if s == 0:
                                    print("Decrement by 1 as all blocks below current tower top equals those in goaltower")
                                    cost = cost - 1
                            
    print("After crtieria 4 cost= ", cost)
    return cost

def moveBlocksHLACost(currentState, goalState):
    if currentState.isGoal(goalState):
        return 0
    
    cost = 0
    #Current state has a blank and goal state has a block. 
    #So increment the heuristic value by the number of blocks needed to be moved into a place
    
    for goalxz, goaltower in goalState.stateMap.items():
        if currentState.numBlksAt(goalxz) == 0:
            cost = cost + goalState.numBlksAt(goalxz)
            
#     print("After crtieria 1 cost= ", cost)        
    #Current state has a block and goal state has a blank. 
    #So increment the heuristic value by the number of blocks needed to be moved out of a place.

    for currentxz, currenttower in currentState.stateMap.items():
        if currentState.numBlksAt(currentxz) > 0 and goalState.numBlksAt(currentxz) == 0:
            cost = cost + currentState.numBlksAt(currentxz)
            
#     print("After crtieria 2 cost= ", cost)
    
    #Both states have a block, but they are not the same. The incorrect block must be moved out 
    #and the correct blocks must be moved in. 
    #Increment the heuristic value by the number of these blocks. 
    
    for goalxz, goaltower in goalState.stateMap.items():
        if currentState.numBlksAtIncludeEmpty(goalxz) > 0:
            currenttower = currentState.towerAt(goalxz)
            currentEmptyCntNeedsFill=0
            currentNonEmptyCntNeedsChange=0
            for goalBlkIdx, goalBlk in enumerate(goaltower):
                if (blksEqual(goalBlk, currenttower[goalBlkIdx]) == False):
                    if currenttower[goalBlkIdx] == '':                        
                        currentEmptyCntNeedsFill = currentEmptyCntNeedsFill + 1
                    else:
                        currentNonEmptyCntNeedsChange = currentNonEmptyCntNeedsChange + 2
            
            cost = cost + currentEmptyCntNeedsFill
            cost = cost + currentNonEmptyCntNeedsChange
            
#     print("After crtieria 3 cost= ", cost)   
    
    #Both states have a block, and they are the same. However, the position above it is incorrect. 
    #Increment the heuristic value by the number of incorrect positions above the correct block.
    
#     for goalxz, goaltower in goalState.stateMap.items():
#         currentTowerBlkCnt = currentState.numBlksAtIncludeEmpty(goalxz)
#         goalTowerBlkCnt = goalState.numBlksAt(goalxz)
#         if (currentTowerBlkCnt > 0 and currentTowerBlkCnt >= goalTowerBlkCnt):            
#             currenttower = currentState.towerAt(goalxz)
            
#             firstunequalidx = next( (i for i, currentBlk in enumerate(currenttower[:goalTowerBlkCnt]) \
#                                  if (not blksEqual(currentBlk, goaltower[i])) ), None)
            
#             if firstunequalidx is not None:
#                 cost = cost + (goalTowerBlkCnt - firstunequalidx) 
                
#     print("After crtieria 4 cost= ", cost)
    
    
    # For a given (nonempty) place, find the top-most block. Look for its position in the goal state. 
   
    for currentxz, currenttower in currentState.stateMap.items():
            currentTowerBlkCnt = currentState.numBlksAt(currentxz)
            if currentTowerBlkCnt > 0:
                currentTowerTopBlk = currenttower[currentTowerBlkCnt-1]
#                 print("currenttower at {} {}".format(currentxz, currenttower))            
                for goalxz, goaltower in goalState.stateMap.items():
#                     print("goaltower at {} {}".format(goalxz, goaltower)) 
                    goalTowerBootomBlk = goaltower[0]
                    goalTowerBlkCnt = goalState.numBlksAt(currentxz)
                    if ( goalxz != currentxz ):
                        if blksEqual(goalTowerBootomBlk, currentTowerTopBlk) :
                        #If tower top item goes on the bottom row and the place is currently empty, the block can be moved there, 
                        #so decrement heuristic by one.
#                             print("Decrement by 1 as top of currenttower equals bottom of goaltower")
                            cost = cost - 1
                        else: 
                        #else if the top-most block doesn't go on bottom row, but all blocks below it in the goal state are 
                        #currently in their correct place, then the block can be moved there, so decrement heuristic by one.
#                             print("currentTowerBlkCnt=", currentTowerBlkCnt)
#                             print("goalTowerBlkCnt=", goalTowerBlkCnt)
                            if (currentTowerBlkCnt > 1):
                                s=sum(1 for i, currentTowerBlk in enumerate(currenttower[:currentTowerBlkCnt-1]) \
                                      if (blksEqual(currentTowerBlk, goaltower[i])==False) )
                                if s == 0:
#                                     print("Decrement by 1 as all blocks below current tower top equals those in goaltower")
                                    cost = cost - 1
                            
#     print("After crtieria 4 cost= ", cost)
    return cost



Overwriting hpdwheuristics.py


# Goal to plans

In [4]:
from hpdwsimulator import *
from hpdwheuristics import *
from itertools import *
import operator
from hla import *

In [5]:
state1 = HPDWSimulator(Xrange=(0,10), Zrange=(0,10), Yrange=(0,20), filename='test1initial.txt', loglevel='silent')
state1

loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=(10, 10, 10)
stateMap:
(0, 0)>['red', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'black', '', '', '', '', '']
(10, 10)>['blue', '', '', '', '', '', '', '', '', '', 'd', '', '', '', '', '', '', '', '', '', '']

In [6]:
goal1 = HPDWSimulator(Xrange=(0,10), Zrange=(0,10), Yrange=(0,20), filename='test1goal.txt', loglevel='silent')
goal1

loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=None
stateMap:
(0, 1)>['black', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'red', '', '', '', '', '']

In [7]:
goal1.fillWCInY()
goal1

loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=None
stateMap:
(0, 1)>['black', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'red']

In [8]:
currentState=state1
currentState.isGoal(goal1)

False

In [9]:
print(moveBlocksHLACost(currentState, goalState=goal1))
for i in range(16):
    hla = MoveBlk(startState=currentState, srcxz=(0,0), destxz=(0,1))
    hla.evalPlans()
    print("endState {} at cost {}".format(hla.endState, hla.cost))
    currentState=hla.endState
    print(moveBlocksHLACost(currentState, goalState=goal1))

32
endState loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=(0, 1, 20)
stateMap:
(0, 1)>['black', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'd']
(0, 0)>['red', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', '', '', '', '', '', '']
(10, 10)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
 at cost 255.0
31
endState loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=(0, 1, 20)
stateMap:
(0, 1)>['black', 'blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'd']
(0, 0)>['red', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', '', '', '', '', '', '', '']
(10, 10)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
 at cost 54.0
29
endState loglevel='s

In [10]:
state2=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,5), filename='test2initial.txt', loglevel='silent')
state2

loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 5) attached=False dronePos=(49, 49, 1)
stateMap:
(50, 50)>['black', '', '', '', '', '']
(49, 49)>['', 'd', '', '', '', '']
(-50, -50)>['red', '', '', '', '', '']

In [11]:
goal2=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,5), filename='test2goal.txt', loglevel='silent')
goal2.fillWCInY()
goal2

loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 5) attached=False dronePos=None
stateMap:
(0, 0)>['red', 'black']

In [12]:
currentState=state2
print(currentState)
print(moveBlocksHLACostDebug(currentState, goalState=goal2))

loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 5) attached=False dronePos=(49, 49, 1)
stateMap:
(50, 50)>['black', '', '', '', '', '']
(49, 49)>['', 'd', '', '', '', '']
(-50, -50)>['red', '', '', '', '', '']

After crtieria 1 cost=  2
After crtieria 2 cost=  4
After crtieria 3 cost=  4
currenttower at (50, 50) ['black', '', '', '', '', '']
goaltower at (0, 0) ['red', 'black']
currentTowerBlkCnt= 1
goalTowerBlkCnt= 0
currenttower at (-50, -50) ['red', '', '', '', '', '']
goaltower at (0, 0) ['red', 'black']
Decrement by 1 as top of currenttower equals bottom of goaltower
After crtieria 4 cost=  3
3


In [13]:
hla1 = MoveBlk(startState=currentState, srcxz=(-50,-50), destxz=(0,0))
hla1.evalPlans()
print("After {} endState \n{}".format(hla1, hla1.endState))
print("cost from here on ", moveBlocksHLACostDebug(hla1.endState, goalState=goal2))

After MoveBlk:(-50, -50)->(0, 0) endState 
loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 5) attached=False dronePos=(0, 0, 5)
stateMap:
(50, 50)>['black', '', '', '', '', '']
(0, 0)>['red', '', '', '', '', 'd']

After crtieria 1 cost=  0
After crtieria 2 cost=  1
After crtieria 3 cost=  2
currenttower at (50, 50) ['black', '', '', '', '', '']
goaltower at (0, 0) ['red', 'black']
currentTowerBlkCnt= 1
goalTowerBlkCnt= 0
currenttower at (0, 0) ['red', '', '', '', '', 'd']
goaltower at (0, 0) ['red', 'black']
After crtieria 4 cost=  2
cost from here on  2


In [14]:
hla2 = MoveBlk(startState=currentState, srcxz=(50,50), destxz=(0,0))
hla2.evalPlans()
print("After {} endState \n{}".format(hla2, hla2.endState))
print("cost from here on ", moveBlocksHLACostDebug(hla2.endState, goalState=goal2))

After MoveBlk:(50, 50)->(0, 0) endState 
loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 5) attached=False dronePos=(0, 0, 5)
stateMap:
(0, 0)>['black', '', '', '', '', 'd']
(-50, -50)>['red', '', '', '', '', '']

After crtieria 1 cost=  0
After crtieria 2 cost=  1
After crtieria 3 cost=  4
currenttower at (0, 0) ['black', '', '', '', '', 'd']
goaltower at (0, 0) ['red', 'black']
currenttower at (-50, -50) ['red', '', '', '', '', '']
goaltower at (0, 0) ['red', 'black']
Decrement by 1 as top of currenttower equals bottom of goaltower
After crtieria 4 cost=  3
cost from here on  3


In [15]:
state3=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,20), filename='test3initial.txt', loglevel='silent')
state3

loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 20) attached=False dronePos=(0, 0, 10)
stateMap:
(-8, 40)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(17, 6)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(21, -11)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(42, 42)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(41, -11)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(-39, -20)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(1, 1)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(-48, 33)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(0, 0)>['red', '', '', '', '', '', '', '', '', '', 'd', '', '', '', '', '', '', '', '', '', '']
(-

In [16]:
goal3=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,20), filename='test3goal.txt', loglevel='silent')
goal3

loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 20) attached=False dronePos=None
stateMap:
('?', '?')>['?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '', '', '', '']

# A*

In [17]:
%%writefile hpdwastar.py
import operator
from hla import *
from hpdwsimulator import *

class Node:
    def __init__(self, state, f=0, g=0 ,h=0, creatorAction=None):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
        self.creatorAction = creatorAction
    def __repr__(self):
        return "Node(" + repr(self.state) + "\ncreatedBy=" + repr(self.creatorAction) + "\nf=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

class Stats:
    def __init__(self, attempts):
        self.attempts = attempts
        
def dronePathSearch(startState, hF, goal, maxAttempts=100, doMaxF = False):
    h = hF(goal, startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    stats = Stats(attempts=maxAttempts)
    return dronePathSearchHelper(startNode, hF, goal, float('inf'), doMaxF, stats)

def dronePathSearchHelper(parentNode, hF, goal, fmax, doMaxF, stats):
    print("Attempts left ", stats.attempts)
    stats.attempts = stats.attempts - 1 
    if stats.attempts == -1:
        return ("failure", float('inf'))
#     print("Current state \n", parentNode.state)
    
    if parentNode.state.dronePos == goal.dronePos: 
        return ([parentNode.state], parentNode.g) # no actions needed as we are desired drone pos now
    
    ## Construct list of children nodes with f, g, and h values
    
    actions = parentNode.state.possibleDroneMoves()
#     print("Possible actions ", actions)
    
    if not actions or len(actions)==0:
        return ("failure", float('inf'))
    children = []
    
    for action in actions:
        (actionStatus, stepCost, childState) = parentNode.state.takeActionImmutable(action)
        # calulate heuristics for child if actionStatus is success
        
        if actionStatus is True:            
            h = hF(goal, childState)            
            g = parentNode.g + stepCost
            if doMaxF:
                f = max(h+g, parentNode.f)
            else:
                f = h+g
            childNode = Node(state=childState, f=f, g=g, h=h, creatorAction=action)           
            children.append(childNode)
        else:
            print("Failed to take action ", action)
    
    while stats.attempts > 0 :
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
#         print("Sorted children \n")
#         for child in children:
#             print("(created by={}, f={}, g={}, h={})".format(child.creatorAction, child.f, child.g, child.h))            
        
        bestChild = children[0]
        if bestChild.f > fmax or bestChild.f == float('inf'):
#             print ("bestChild.f={} > fmax={}".format(bestChild.f, fmax))
            return ("failure", bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result, bestChild.f = dronePathSearchHelper(bestChild, hF, goal, min(fmax,alternativef), doMaxF, stats)
        if result is not "failure":               
            result.insert(0, parentNode.state)     
            return (result, bestChild.f)      
    return ("failure", float('inf'))

def getPossibleTowers(currentState, goalState):    
    possibleTowers = []
    goalCopy = deepcopy(goalState)
    goalCopy.fillWC()
    for goalxz, goaltower in goalState.stateMap.items():
        if goalxz == ('?', '?'):
            # find best destination
            towersWithBlock = deepcopy(currentState.towersWithBlks())
            towersWithBlock.sort(key = lambda n: currentState.numBlksAt(n[0])) # sort
            possibleTowers.append((towersWithBlock[::-1][0][0], goalCopy.stateMap[towersWithBlock[::-1][0][0]]))
        else:
            possibleTowers.append((goalxz, goaltower))
    return possibleTowers

def getPossibleBlkMoves1(currentState, goalState): 
    hlas = []
    for xz, xzTower in currentState.stateMap.items():
        xzBlkCnt = sum(1 for item in xzTower if item != '' and item !='d')
        if (xzBlkCnt > 0 ):
            for destxz in currentState.getPossibleXZs():
                if destxz != xz:
                    hla = MoveBlk(startState=currentState, srcxz=xz, destxz=destxz)
                    hlas.append(hla)    
    return hlas

def getPossibleBlkMoves(currentState, goalState): 
    hlas = []
    for xz, xzTower in currentState.stateMap.items():
        xzBlkCnt = sum(1 for item in xzTower if item != '' and item !='d')
        if (xzBlkCnt > 0 ):
            for destxz, desttower in goalState.stateMap.items():
                if destxz != xz:
                    hla = MoveBlk(startState=currentState, srcxz=xz, destxz=destxz)
                    hlas.append(hla)    
    return hlas

def getPossibleBlkMovesFromPossibleTowers(currentState, goalState): 
    hlas = []
    for xz, xzTower in currentState.stateMap.items():
        xzBlkCnt = sum(1 for item in xzTower if item != '' and item !='d')
        if (xzBlkCnt > 0 ):
            for (destxz, desttower) in getPossibleTowers(currentState, goalState) :
                if destxz != xz:
                    hla = MoveBlk(startState=currentState, srcxz=xz, destxz=destxz)
                    hlas.append(hla)    
    return hlas

def blkMovesSearch(startState, hF, goalState, maxAttempts=100, doMaxF = False):
    goal=goalState.fillWCWithPossibleTowers(getPossibleTowers(startState, goalState))
    h = hF(currentState=startState, goalState=goal)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    stats = Stats(attempts=maxAttempts)
    return blkMovesSearchHelper(startNode, hF, goal, float('inf'), doMaxF, stats)

def blkMovesSearchHelper(parentNode, hF, goal, fmax, doMaxF, stats):
    print("Attempts left ", stats.attempts)
    stats.attempts = stats.attempts - 1 
    if stats.attempts == -1:
        return ("failure", float('inf'))
#     print("Current state \n", parentNode.state)
    
    if parentNode.state.isGoal(goal):
        return ([parentNode.state], parentNode.g) # no actions needed
    ## Construct list of children nodes with f, g, and h values
   
    blkMoveHlas = getPossibleBlkMovesFromPossibleTowers(currentState=parentNode.state, goalState=goal)
#     print("Possible moves \n", blkMoveHlas)
    
    if not blkMoveHlas or len(blkMoveHlas)==0:
        return ("failure", float('inf'))
    children = []    
    for blkMoveHla in blkMoveHlas:
        blkMoveHla.evalPlans()
        childState = blkMoveHla.endState
        stepCost = 0
        # calulate heuristics for child 
        h = hF(currentState=childState, goalState=goal)            
        g = parentNode.g + stepCost
        if doMaxF:
            f = max(h+g, parentNode.f)
        else:
            f = h+g
        childNode = Node(state=childState, f=f, g=g, h=h, creatorAction=blkMoveHla)           
        children.append(childNode)       
    
    while stats.attempts > 0 :
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
#         print ("bestChild.f={} > fmax={}".format(bestChild.f, fmax))
        if bestChild.f > fmax or bestChild.f == float('inf'):
            return ("failure", bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result, bestChild.f = blkMovesSearchHelper(bestChild, hF, goal, min(fmax,alternativef), doMaxF, stats)
        if result is not "failure":               
            result.insert(0, parentNode.state)     
            return (result, bestChild.f)      
    return ("failure", float('inf'))




Overwriting hpdwastar.py


In [18]:
from hpdwsimulator import *
from hpdwastar import *
from hpdwheuristics import *
from copy import *
import time as time

In [19]:
state1=HPDWSimulator(Xrange=(0,10), Zrange=(0,10), Yrange=(0,20), filename='test1initial.txt', loglevel='silent')
state1

loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=(10, 10, 10)
stateMap:
(0, 0)>['red', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'black', '', '', '', '', '']
(10, 10)>['blue', '', '', '', '', '', '', '', '', '', 'd', '', '', '', '', '', '', '', '', '', '']

In [20]:
goal1=HPDWSimulator(Xrange=(0,10), Zrange=(0,10), Yrange=(0,20), filename='test1goal.txt', loglevel='silent')
goal1

loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=None
stateMap:
(0, 1)>['black', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'red', '', '', '', '', '']

In [21]:
goal1.fillWCInY()
goal1

loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=None
stateMap:
(0, 1)>['black', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'red']

In [22]:
before=time.time()
result = blkMovesSearch(state1, moveBlocksHLACost, goal1)
after=time.time()
print("Time taken", after-before)


Attempts left  100
Attempts left  99
Attempts left  98
Attempts left  97
Attempts left  96
Attempts left  95
Attempts left  94
Attempts left  93
Attempts left  92
Attempts left  91
Attempts left  90
Attempts left  89
Attempts left  88
Attempts left  87
Attempts left  86
Attempts left  85
Attempts left  84
Time taken 0.031200647354125977


In [23]:
print(len(result[0]))
print(result[0])

17
[loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=(10, 10, 10)
stateMap:
(0, 0)>['red', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'black', '', '', '', '', '']
(10, 10)>['blue', '', '', '', '', '', '', '', '', '', 'd', '', '', '', '', '', '', '', '', '', '']
, loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=(0, 1, 20)
stateMap:
(0, 1)>['black', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'd']
(0, 0)>['red', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', '', '', '', '', '', '']
(10, 10)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
, loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=(0, 1, 20)
stateMap:
(0, 1)>['black', 'blue', '', '', '', '', '', '', '

In [24]:
state2=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,5), filename='test2initial.txt', loglevel='silent')
state2

loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 5) attached=False dronePos=(49, 49, 1)
stateMap:
(50, 50)>['black', '', '', '', '', '']
(49, 49)>['', 'd', '', '', '', '']
(-50, -50)>['red', '', '', '', '', '']

In [25]:
goal2=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,5), filename='test2goal.txt', loglevel='silent')
goal2.fillWCInY()
goal2

loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 5) attached=False dronePos=None
stateMap:
(0, 0)>['red', 'black']

In [28]:
before=time.time()
result = blkMovesSearch(state2, moveBlocksHLACost, goal2)
after=time.time()
print("Time taken", after-before)
print(len(result[0]))
print(result[0])

Attempts left  100
Attempts left  99
Attempts left  98
Time taken 0.0025000572204589844
3
[loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 5) attached=False dronePos=(49, 49, 1)
stateMap:
(50, 50)>['black', '', '', '', '', '']
(49, 49)>['', 'd', '', '', '', '']
(-50, -50)>['red', '', '', '', '', '']
, loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 5) attached=False dronePos=(0, 0, 5)
stateMap:
(50, 50)>['black', '', '', '', '', '']
(0, 0)>['red', '', '', '', '', 'd']
, loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 5) attached=False dronePos=(0, 0, 5)
stateMap:
(0, 0)>['red', 'black', '', '', '', 'd']
]


In [29]:
state3=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,20), filename='test3initial.txt', loglevel='silent')
state3

loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 20) attached=False dronePos=(0, 0, 10)
stateMap:
(-8, 40)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(17, 6)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(21, -11)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(42, 42)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(41, -11)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(-39, -20)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(1, 1)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(-48, 33)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(0, 0)>['red', '', '', '', '', '', '', '', '', '', 'd', '', '', '', '', '', '', '', '', '', '']
(-

In [30]:
goal3=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,20), filename='test3goal.txt', loglevel='silent')
goal3.fillWCInY()
goal3

loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 20) attached=False dronePos=None
stateMap:
('?', '?')>['*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']

In [31]:
before=time.time()
result = blkMovesSearch(state3, moveBlocksHLACost, goal3)
after=time.time()
print("Time taken", after-before)
print(len(result[0]))
print(result[0])

Attempts left  100
Attempts left  99
Attempts left  98
Attempts left  97
Attempts left  96
Attempts left  95
Attempts left  94
Attempts left  93
Attempts left  92
Attempts left  91
Attempts left  90
Attempts left  89
Attempts left  88
Attempts left  87
Attempts left  86
Attempts left  85
Time taken 0.18720364570617676
16
[loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 20) attached=False dronePos=(0, 0, 10)
stateMap:
(-8, 40)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(17, 6)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(21, -11)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(42, 42)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(41, -11)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(-39, -20)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', ''

In [32]:
print(len(result[0]))
print(result[0])

16
[loglevel='silent' Xrange=(-50, 50) Zrange=(-50, 50) Yrange=(0, 20) attached=False dronePos=(0, 0, 10)
stateMap:
(-8, 40)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(17, 6)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(21, -11)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(42, 42)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(41, -11)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(-39, -20)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(1, 1)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(-48, 33)>['white', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
(0, 0)>['red', '', '', '', '', '', '', '', '', '', 'd', '', '', '', '', '', '', '', '', '', ''

In [33]:
destTowerXZ = (0,0)
destTowerHt = sum(1 for item in state1.stateMap[destTowerXZ] if item != '')
droneDest = (destTowerXZ[0], destTowerXZ[1], destTowerHt)
HLA_move_goal_1_1 = state1.moveDroneImmutable((droneDest))
HLA_move_goal_1_1

loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=(0, 0, 16)
stateMap:
(0, 0)>['red', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'black', 'd', '', '', '', '']
(10, 10)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

In [34]:
before=time.time()
result = dronePathSearch(state1, moveDroneCost, HLA_move_goal_1_1)
after=time.time()
print(after-before)

Attempts left  100
Attempts left  99
Attempts left  98
Attempts left  97
Attempts left  96
Attempts left  95
Attempts left  94
Attempts left  93
Attempts left  92
Attempts left  91
Attempts left  90
0.031200647354125977


In [35]:
print("Path Length=", len(result[0])-1)
print()
for state in result[0]:
    print(state)

Path Length= 10

loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=(10, 10, 10)
stateMap:
(0, 0)>['red', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'black', '', '', '', '', '']
(10, 10)>['blue', '', '', '', '', '', '', '', '', '', 'd', '', '', '', '', '', '', '', '', '', '']

loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=(9, 9, 11)
stateMap:
(0, 0)>['red', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'white', 'blue', 'black', '', '', '', '', '']
(9, 9)>['', '', '', '', '', '', '', '', '', '', '', 'd', '', '', '', '', '', '', '', '', '']
(10, 10)>['blue', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

loglevel='silent' Xrange=(0, 10) Zrange=(0, 10) Yrange=(0, 20) attached=False dronePos=(8, 8, 12)
stateMap:
(8, 8)>['', '', '', '', '', '', '', '',

In [36]:
%%writefile experiments.py
from hpdwsimulator import *
from hpdwastar import *
from hpdwheuristics import *
from copy import *
from hla import *
import time as time

state1=HPDWSimulator(Xrange=(0,10), Zrange=(0,10), Yrange=(0,20), filename='test1initial.txt', loglevel='silent')
print(state1)
goal1=HPDWSimulator(Xrange=(0,10), Zrange=(0,10), Yrange=(0,20), filename='test1goal.txt', loglevel='silent')
goal1.fillWCInY()
print(goal1)
before=time.time()
result = blkMovesSearch(state1, moveBlocksHLACost, goal1)
after=time.time()
print("Time Taken ", after-before)
print(len(result[0]))
print(result[0])

state2=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,5), filename='test2initial.txt', loglevel='silent')
print(state2)
goal2=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,5), filename='test2goal.txt', loglevel='silent')
goal2.fillWCInY()
print(goal2)
before=time.time()
result = blkMovesSearch(state2, moveBlocksHLACost, goal2)
after=time.time()
print("Time Taken ", after-before)
print(len(result[0]))
print(result[0])

state3=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,20), filename='test3initial.txt', loglevel='silent')
print(state3)
goal3=HPDWSimulator(Xrange=(-50,50), Zrange=(-50,50), Yrange=(0,20), filename='test3goal.txt', loglevel='silent')
goal3.fillWCInY()
print(goal3)
before=time.time()
result = blkMovesSearch(state3, moveBlocksHLACost, goal3)
after=time.time()
print("Time Taken ", after-before)
print(len(result[0]))
print(result[0])

#### Drone moves
destTowerXZ = (0,0)
destTowerHt = sum(1 for item in state1.stateMap[destTowerXZ] if item != '')
droneDest = (destTowerXZ[0], destTowerXZ[1], destTowerHt)
HLA_move_goal_1_1 = state1.moveDroneImmutable((droneDest))
HLA_move_goal_1_1

result = dronePathSearch(state1, moveDroneCost, HLA_move_goal_1_1)

print("Path Length=", len(result[0])-1)
print()
for state in result[0]:
    print(state)

Overwriting experiments.py
